##### 【 모델 성능 개선 - 튜닝 】
- scikit-learn에서는 튜닝을 위한 클래스 제공
    * GridSearchCV
    * RandomizedSearchCV
    * CV 즉, 교차검증 함께 진행
    * 시간이 오래 걸림!!

[1] 모듈 로딩 및 데이터 준비 <hr>

In [2]:
## [1-1] 모듈로딩
## 기본 모듈
import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib

## ML 관련 모듈
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV  ## 튜닝 관련
from sklearn.neighbors import KNeighborsClassifier                    ## 학습 알고리즘
from sklearn.model_selection import train_test_split                  ## 데이터셋 관련
from sklearn.preprocessing import StandardScaler                      ## 피쳐 스케일러 모듈

In [3]:
## [1-2] 데이터 준비
DATA_FILE = '../DATA/iris.csv'

irisDF = pd.read_csv(DATA_FILE)

In [4]:
## [1-3] 데이터 기본 정보 확인
irisDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


[2] 데이터 전처리 <hr>
- 기본 데이터 전처리 : 결측치, 중복값, 피쳐별 분포, 이상치
- 학습관련 분석 : 학습 알고리즘에 따른 처리, 피쳐와 타겟, 피쳐와 피쳐
- 학습관련 분리 : 피쳐와 타겟 분리, 학습용/테스트용 분리
- 학습데이터 전처리 : 이상치, 스케일러, 인코딩 ...
- 검증/테스트용 데이터 : 학습 알고리즘에 대입 하기위한 형태 맞춤 진행 
    * 데이터에 대한 도메인 지식 => ★이상치 그대로 / 변경 여부 선택

In [7]:
## ========================================================================
## [2-1] 피쳐와 타겟 분리
## ========================================================================
featureDF = irisDF.drop("variety", axis=1)
targetSR = irisDF['variety']

## Train / Test 분리 (검증은 CV로 처리)
x_train, x_test, y_train, y_test = train_test_split(featureDF, targetSR,    ## 입력, 정답
                                                    test_size=0.2,          ## 20%는 테스트용 -> 80%는 학습용으로 자동으로 설정
                                                    random_state=42,        ## 랜덤 시드 번호 -> 항상 똑같이 나눠주도록 고정하는거임
                                                    stratify=targetSR)      ## Train/Test에 각 클래스 비율을 동일하게 유지

In [ ]:
## ========================================================================
## # [2-2] 학습 알고리즘을 위한 전처리 : 거리기반 알고리즘스케일러
## ========================================================================
# 스케일러 인스턴스 생성
scaler = StandardScaler()

# Train에 대해서만 fit
x_train_scaled = scaler.fit_transform(x_train)  ## train만 fit, 즉 학습해야함

# Test는 transform만
x_test_scaled = scaler.transform(x_test)        ## test는 스케일만 해줘야함

[3] 학습 및 검증, 하이퍼파라미터 찾기 <hr>
- GridSearchCV : 모든 파라미터 조합으로 모델 생성 및 학습/검증 진행

In [9]:
# 1) 기본 모델
knn = KNeighborsClassifier()

# 2) 그리드 탐색용 하이퍼파라미터 설정
#       키 -> 학습 알고리즘의 매개변수 즉, 속성명
#       값 -> 학습 알고리즘의 매개변수 즉, 속성에 적용할 수 있는 값들
param_grid = {"n_neighbors" : [1, 3, 5, 7, 9, 11, 13, 15],
              "weights" : ["uniform", 'distance'],
              "p" : [1,2]   # 1: 맨해튼 거리, 2: 유클리드 거리
            } 

# 3) GridSearchCV 설정
grid_search = GridSearchCV(estimator=knn,
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=1                ## 진행상황에 출력할 수 있게끔해줌
                           )

# 4) 학습 (★반드시 Train 데이터만 사용)
grid_search.fit(x_train_scaled, y_train)

# => 학습 즉, fit() 이후 모델 파라미터(파라미터이름_)들
print("▶ GridSearchCV 최적 하이퍼파라미터:", grid_search.best_params_)
print("▶ GridSearchCV CV 최고 정확도:", grid_search.best_score_)

resultDF = pd.DataFrame(grid_search.cv_results_)
print("▶ 교차검증 결과:")
print(resultDF)

# 5) 최적 모델로 Test 성능 평가
best_knn_grid = grid_search.best_estimator_
y_pred_grid = best_knn_grid.predict(x_test_scaled)

print("\n▶ Test Accuracy (GridSearch 최적 모델) :", best_knn_grid.score(x_test_scaled, y_test))

Fitting 5 folds for each of 32 candidates, totalling 160 fits
▶ GridSearchCV 최적 하이퍼파라미터: {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'}
▶ GridSearchCV CV 최고 정확도: 0.9666666666666668
▶ 교차검증 결과:
    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0        0.010655      0.010811         0.005910        0.002148   
1        0.003755      0.000975         0.004735        0.000871   
2        0.004159      0.001478         0.005196        0.000986   
3        0.002951      0.002386         0.002934        0.000546   
4        0.001947      0.001067         0.003589        0.001474   
5        0.002308      0.000611         0.003617        0.000836   
6        0.018975      0.011786         0.006201        0.002301   
7        0.002251      0.000716         0.004672        0.003804   
8        0.002021      0.000010         0.003730        0.000878   
9        0.002330      0.000633         0.002346        0.000781   
10       0.002557      0.001004         0.004166        0.

<hr>  

- **RandomizedSearchCV:GridSearchCV의 단점인 시간 개선 튜닝 방법**

In [10]:
## 랜덤 모듈
from scipy.stats import randint

# 1) 기본 모델
knn = KNeighborsClassifier()

# 2) 랜덤 탐색용 분포 설정
param_dist = {"n_neighbors" : randint(1,31),        # 1 ~ 30 사이의 정수
              "weights" : ["uniform", 'distance'],
              "p" : [1,2]   
            }

# 3) RandomizedSearchCV 설정
random_search = RandomizedSearchCV(estimator=knn,                 ## 어떤 모델로 튜닝할지
                                 param_distributions=param_dist,  ## 
                                 n_iter = 20,                     ## 랜덤 조합의 개수 의미
                                 cv=5,                            ## 5-Fold 교차검증
                                 scoring='accuracy',              ## 정확도(accuracy) 기준으로 “좋고 나쁨” 평가.
                                 n_jobs=-1,                       ## 사용 가능한 CPU 코어를 모두 사용 -> 병렬처리
                                 random_state=42,
                                 verbose=2                        ## 진행 상황을 얼마나 자세히 출력 ex] 0이면 조용, 1이면 간단 로그, 2면 Fold마다 꽤 자세히
                                 )

# 4) 학습 (★역시 Train 데이터만 사용)
random_search.fit(x_train_scaled, y_train)

print("▶ RandomizedSearchCV 최적 하이퍼파라미터:", random_search.best_params_)
print("▶ RandomizedSearchCV CV 최고 정확도:", random_search.best_score_)


# 5) 최적 모델로 Test 성능 평가
best_knn_rand = random_search.best_estimator_

print("\n▶ Test Accuracy (RandomizedSearchCV 최적 모델) :", best_knn_rand.score(x_test_scaled, y_test))

Fitting 5 folds for each of 20 candidates, totalling 100 fits
▶ RandomizedSearchCV 최적 하이퍼파라미터: {'n_neighbors': 24, 'p': 1, 'weights': 'distance'}
▶ RandomizedSearchCV CV 최고 정확도: 0.975

▶ Test Accuracy (RandomizedSearchCV 최적 모델) : 0.9333333333333333
